In [4]:
import psycopg2

conn = psycopg2.connect(
    dbname="ragdb",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

In [5]:
import pandas as pd

df = pd.read_csv("/home/ys0660/25-1/Demo/data/details_2025-05-22.csv")  # 파일 경로에 맞게 수정

In [6]:
from datetime import datetime

# 날짜 문자열을 datetime 객체로 변환
df["start_date"] = df["start_date"].where(df["start_date"].notna(), None)
df["end_date"] = df["end_date"].where(df["end_date"].notna(), None)

In [7]:
cur.execute("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'festivals'
    ORDER BY ordinal_position;
""")
columns = cur.fetchall()
for col in columns:
    print(col[0])


In [10]:
df.head()

,name,location,start_date,end_date,description,contents,homepage,poster
0,단양 소백산 철쭉제,충청북도 단양군 단양읍 수변로 38,2025.05.22,2025.05.25,단양 소백산철쭉제는 매 년 5월에 개최된다. 5월 소백산 정상에 핀 철쭉을 즐기며 ...,"1.공연프로그램: 불꽃을 활용한 레이져쇼,동춘서커스, 개막콘서트(이찬원, 클라씨,왁...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
1,밀양아리랑대축제,경상남도 밀양시 삼문동 1-1,2025.05.22,2025.05.25,밀양아리랑대축제는 영남루 대보수사업과 함께 시작된 축제로 현재는 밀양아리랑의 현대적...,1. 대표프로그램- 2025 밀양강오딧세이 : 밀양의 최고 절경인 영남루와 밀양강을...,https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
2,원주용수골꽃양귀비축제,강원특별자치도 원주시 용수골길 311 꽃양귀비축제장,2025.05.22,2025.06.08,"자연경관이 뛰어난 마을에 매료되어, 귀농한 주민이 꽃을 너무 좋아해서 2005년 3...","대표프로그램1. 차분한 꽃 관람2. 체험 프로그램 : 꽃양귀비티셔츠 체험, 깡통열차...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
3,공공한옥 밤마실,서울특별시 종로구 계동길 37 (계동) 북촌문화센터,2025.05.21,2025.05.31,"도심 속 한옥에 드리운 청명한 기운을 느낄 수 있는 초하(初夏)의 계절, 북촌문화센...","1. 전시: 빛 스민 집 展(강현지, 정성아), 밤을 지키는 불빛 展(이승정)2. ...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...
4,울산대공원 장미축제,울산광역시 남구 남부순환도로 377 (옥동),2025.05.21,2025.05.25,울산대공원 장미원에서 개최되는 울산대공원 장미축제는 국내 최고의 장미원에서 피어나는...,"1. 메인프로그램 : 개막식(퍼레이드, 개막 점등식, 불꽃쇼, 개막공연)2. 공연프...",https://korean.visitkorea.or.kr/kfes/detail/fs...,https://kfescdn.visitkorea.or.kr/kfes/upload/c...


In [46]:
columns_to_drop = [
    "host", "organizer", "address_road", "address_lot",
    "latitude", "longitude", "created_at", "region"
]

# 각 컬럼에 대해 DROP COLUMN 실행
for col in columns_to_drop:
    try:
        cur.execute(f"ALTER TABLE festivals DROP COLUMN IF EXISTS {col};")
        print(f"Dropped column: {col}")
    except Exception as e:
        print(f"Error dropping column {col}: {e}")
        conn.rollback()
    else:
        conn.commit()

Dropped column: host
Dropped column: organizer
Dropped column: address_road
Dropped column: address_lot
Dropped column: latitude
Dropped column: longitude
Dropped column: created_at
Dropped column: region


In [12]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS festivals (
        id SERIAL PRIMARY KEY,
        name TEXT,
        location TEXT,
        start_date DATE,
        end_date DATE,
        description TEXT,
        contents TEXT,
        homepage TEXT,
        poster TEXT
    )
""")
conn.commit()

In [14]:
for _, row in df.iterrows():
    try:
        cur.execute("""
            INSERT INTO festivals (
                name, location, start_date, end_date,
                description, contents, homepage, poster
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row["name"],
            row["location"],
            row["start_date"],
            row["end_date"],
            row["description"],
            row["contents"],
            row["homepage"],
            row["poster"]
        ))
    except Exception as e:
        print(f"Error inserting row: {row['name']}, error: {e}")
        conn.rollback()  # 이 행의 실패로 인한 트랜잭션 초기화

Error inserting row: 마토예술제, error: invalid input syntax for type date: "~"
LINE 5: ... 평택시 팽성읍 안정쇼핑로 11 안정리 예술인광장', '~', '~', ...
                                                            ^

Error inserting row: 마한역사문화제, error: invalid input syntax for type date: "~"
LINE 5: ...'전라남도 영암군 시종면 옥야리 526 마한문화공원', '~', '~', ...
                                                             ^

Error inserting row: 물금벚꽃축제, error: invalid input syntax for type date: "~"
LINE 5: ...경상남도 양산시 물금읍 황산공원1길 275 황산공원', '~', '~', ...
                                                             ^

Error inserting row: 비슬산 참꽃문화제, error: invalid input syntax for type date: "~"
LINE 5: ...군 유가읍 일연선사길 10 비슬산유스호스텔 주차장', '~', '~', ...
                                                             ^

Error inserting row: 안동벚꽃축제, error: invalid input syntax for type date: "~"
LINE 5: ...꽃축제', '경상북도 안동시 축제장길 200 (운흥동)', '~', '~', ...
                                                             ^

Error inserting ro

In [15]:
cur.execute("SELECT * FROM festivals LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

(1955, '포천시 청년축제', '경기도 포천시 호국로 1518', datetime.date(2024, 10, 12), datetime.date(2024, 10, 13), '2024년 제5회 포천시 청년축제는 "청년, 그리고 울림" 이라는 슬로건으로 청년들이 직접 기획하고 준비한 축제이다. 청년들이 즐길수 있는 다양한 체험부스와 청년예술인 공연, 연예인축하공연, 청년대상 시상등 볼거리, 즐길거리등이 준비되어 있다.', '1. 메인프로그램 : 청년동아리 공연, 기념식, 연예인축하공연2. 부대프로그램 : 체험부스, 판매부스3. 소비자 참여 프로그램 : 청년노래방 "일단불러"4. 기타 내용 : 포토존 운영', 'https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=22b7e854-7ed2-41c6-819f-6a340e4249aa&cntntsNm=%ED%8F%AC%EC%B2%9C%EC%8B%9C%EC%B2%AD%EB%85%84%EC%B6%95%EC%A0%9C', 'https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/22b7e854-7ed2-41c6-819f-6a340e4249aa_3.jpg')
(2020, '청라 와인 페스티벌', '인천광역시 서구 청라대로 88 (청라동)', datetime.date(2024, 10, 5), datetime.date(2024, 10, 6), '2024 청라 와인 페스티벌은 10월 5일 토요일부터 10월 6일 일요일까지 청라호수공원 플라워아일랜드에서 진행된다. 이번 축제는 남미의 분위기를 느낄 수 있는 문화예술 공연 및 프로그램으로 구성된다. 축제 1일 차에는 지역예술가의 음악과 남미 공연을 진행하며, 축제 2일 차에는 뮤지컬 배우 카이, 가수 효린, 하현상, VOS 공연 및 레이저쇼도 진행된다. 양일간 나이 제한 없이 전 연령 참여가 가능한 체험프로그램도 운영된다.', '1. 메인프로그램1) 공연프로그램 (

In [16]:
cur.execute("SELECT COUNT(*) FROM festivals;")
count = cur.fetchone()[0]
print(f"총 데이터 수: {count}행")

총 데이터 수: 1016행


In [4]:
# 직접 하나만 넣고 싶을 때
single_data = {
    "name": "부산국제록페스티벌",
    "location": "부산광역시 사상구 삼락동 29-50",
    "start_date": "2025-09-26",
    "end_date": "2025-09-28",
    "description": """*하기 축제 먹거리 정보는 전년도(2024년) 정보로, 2025년도 축제 먹거리 정보는 업데이트 중에 있습니다.*

    부산국제록페스티벌은 2000년 첫 개최 이래, 26년 동안 명맥을 꾸준히 지켜온 국내 최초·최장수 국제록페스티벌이다. 2025년에는 9월 26일(금)부터 9월 28일(일)까지 3일간, 삼락생태공원에서 개최되며, 다양한 국내외 아티스트가 참여할 예정이다.공원 곳곳에 조성된 무대에서 신진 아티스트부터 글로벌 최정상급 헤드라이너까지 취향에 따라 다양한 공연을 관람할 수 있으며,관람객 참여형 게임 이벤트, 푸드코트 라운지, 캠핑존 등 다양한 부대 이벤트도 즐길 수 있다.""",
    "contents": "1. 메인프로그램 : 부산국제록페스티벌\n2. 부대프로그램 : Rookies on the BU-ROCK, Road to BU-ROCK 등\n3. 기타 : MD부스, 푸드코트 라운지, 캠핑존 등",
    "homepage": "https://korean.visitkorea.or.kr/kfes/detail/fstvlDetail.do?fstvlCntntsId=8146cbf1-2012-4178-aa12-06ddfb702361&cntntsNm=%EB%B6%80%EC%82%B0%EA%B5%AD%EC%A0%9C%EB%A1%9D%ED%8E%98%EC%8A%A4%ED%8B%B0%EB%B2%8C",
    "poster": "https://kfescdn.visitkorea.or.kr/kfes/upload/contents/db/8146cbf1-2012-4178-aa12-06ddfb702361_3.png"
}


try:
    cur.execute("""
        INSERT INTO festivals (
            name, location, start_date, end_date,
            description, contents, homepage, poster
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        single_data["name"],
        single_data["location"],
        single_data["start_date"],
        single_data["end_date"],
        single_data["description"],
        single_data["contents"],
        single_data["homepage"],
        single_data["poster"]
    ))
    conn.commit()  # 성공하면 커밋
except Exception as e:
    print(f"Error inserting data: {single_data['name']}, error: {e}")
    conn.rollback()  # 실패 시 롤백

cur.execute("UPDATE festivals SET updated_at = NOW();")

In [17]:
conn.commit()
cur.close()
conn.close()